In [21]:
import pandas as pd
import requests
import os

# URL for age file
url = 'https://www.abs.gov.au/statistics/people/population/regional-population-age-and-sex/2023/32350DS0002_2023.xlsx'

# download
save_path = os.path.expanduser('../data/raw/external/32350DS0002_2023.xlsx')
response = requests.get(url)
with open(save_path, 'wb') as file:
    file.write(response.content)

# Read "Table 1" from Excel file
age_file = pd.read_excel(save_path, sheet_name="Table 1")

# Due to the problem in format, swap rows 5 and 6 (data after column 11)
age_file.iloc[[3, 4], 10:] = age_file.iloc[[4, 3], 10:].values

swapped_file_path = '../data/raw/external/swapped_rows.csv'
age_file.to_csv(swapped_file_path, index=False)

# Read the swapped file and skip the first empty five lines
a_file = pd.read_csv(swapped_file_path, skiprows=5)

# Reads another file for merging
b_file_path = './data/curated/external/cleaned_final_data.csv'
b_file = pd.read_csv(b_file_path)

# choose needed columns in a_file ：SA2 code, Median age, People aged 0-14 years, People aged 15-64 years, People aged 65 years and over
age_file_subset = a_file[['SA2 code', 'Median age', 'People aged 0-14 years', 'People aged 15-64 years', 'People aged 65 years and over']]

# Merge data based on the SA2_CODE_2021 column of the b file and the SA2 code column of the a file
merged_file = b_file.merge(age_file_subset, how='left', left_on='SA2_CODE_2021', right_on='SA2 code')

# Convert specific column floating-point numbers and process NaN values
columns_to_convert = ['Median age', 'People aged 0-14 years', 'People aged 15-64 years', 'People aged 65 years and over']
merged_file[columns_to_convert] = merged_file[columns_to_convert].apply(pd.to_numeric, errors='coerce').fillna(0)

final_file_path = '../data/curated/external/final_file.csv'
merged_file.to_csv(final_file_path, index=False)



所有操作成功完成，文件已保存到: /Users/fenglingyi/Downloads/final_file.csv
